In [49]:
#Dependencies
import pandas as pd
import csv
import matplotlib
import seaborn
import os
import numpy as np
import time
import json
from config import wolf_api_key
from pprint import pprint
import requests


In [50]:
#import csv into dataframe and preview
cities_df = pd.read_csv("500_Cities_original.csv", encoding="ISO-8859-1")

In [51]:
#Keep only city-level rows
cities_df = cities_df.loc[cities_df["GeographicLevel"] == "City"]

In [52]:
#keep only age-adjusted data values
#Keep only city-level rows
cities_df = cities_df.loc[cities_df["DataValueTypeID"] == "AgeAdjPrv"]
cities_df.head()

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
56,2016,AL,Alabama,Birmingham,City,BRFSS,Prevention,0107000,Current lack of health insurance among adults ...,%,...,19.9,NaN,NaN,212237,"(33.5275663773, -86.7988174678)",PREVENT,ACCESS2,107000.0,NaN,Health Insurance
156,2016,AL,Alabama,Birmingham,City,BRFSS,Health Outcomes,0107000,Arthritis among adults aged >=18 Years,%,...,29.4,NaN,NaN,212237,"(33.5275663773, -86.7988174678)",HLTHOUT,ARTHRITIS,107000.0,NaN,Arthritis
256,2016,AL,Alabama,Birmingham,City,BRFSS,Unhealthy Behaviors,0107000,Binge drinking among adults aged >=18 Years,%,...,13.1,NaN,NaN,212237,"(33.5275663773, -86.7988174678)",UNHBEH,BINGE,107000.0,NaN,Binge Drinking
359,2015,AL,Alabama,Birmingham,City,BRFSS,Health Outcomes,0107000,High blood pressure among adults aged >=18 Years,%,...,44.2,NaN,NaN,212237,"(33.5275663773, -86.7988174678)",HLTHOUT,BPHIGH,107000.0,NaN,High Blood Pressure
459,2015,AL,Alabama,Birmingham,City,BRFSS,Prevention,0107000,Taking medicine for high blood pressure contro...,%,...,70.2,NaN,NaN,212237,"(33.5275663773, -86.7988174678)",PREVENT,BPMED,107000.0,NaN,Taking BP Medication


In [53]:
#trim down columns
cities_cleaned_df = cities_df.loc[:, ["Year", "StateDesc", "CityName",
                                    "Category", "Measure", "DataValueTypeID", "Data_Value", "PopulationCount",
                                      "Count_of_Individuals", "GeoLocation", "MeasureId", "CityFIPS",
                                      "Short_Question_Text"]]
cities_cleaned_df.head(6)

C:\Users\maste\anaconda\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,Year,StateDesc,CityName,Category,Measure,DataValueTypeID,Data_Value,PopulationCount,Count_of_Individuals,GeoLocation,MeasureId,CityFIPS,Short_Question_Text
56,2016,Alabama,Birmingham,Prevention,Current lack of health insurance among adults ...,AgeAdjPrv,19.5,212237,NaN,"(33.5275663773, -86.7988174678)",ACCESS2,107000.0,Health Insurance
156,2016,Alabama,Birmingham,Health Outcomes,Arthritis among adults aged >=18 Years,AgeAdjPrv,29.3,212237,NaN,"(33.5275663773, -86.7988174678)",ARTHRITIS,107000.0,Arthritis
256,2016,Alabama,Birmingham,Unhealthy Behaviors,Binge drinking among adults aged >=18 Years,AgeAdjPrv,13.0,212237,NaN,"(33.5275663773, -86.7988174678)",BINGE,107000.0,Binge Drinking
359,2015,Alabama,Birmingham,Health Outcomes,High blood pressure among adults aged >=18 Years,AgeAdjPrv,44.1,212237,NaN,"(33.5275663773, -86.7988174678)",BPHIGH,107000.0,High Blood Pressure
459,2015,Alabama,Birmingham,Prevention,Taking medicine for high blood pressure contro...,AgeAdjPrv,70.1,212237,NaN,"(33.5275663773, -86.7988174678)",BPMED,107000.0,Taking BP Medication
560,2016,Alabama,Birmingham,Health Outcomes,Cancer (excluding skin cancer) among adults ag...,AgeAdjPrv,5.7,212237,NaN,"(33.5275663773, -86.7988174678)",CANCER,107000.0,Cancer (except skin)


In [54]:
just_cities = cities_df.loc[:, ["StateDesc","CityName","PopulationCount","GeoLocation","CityFIPS"]]
just_cities = just_cities.drop_duplicates()

# Make a small dataframe to play around with
just_cities_head = just_cities.head(3)
just_cities.head()

,StateDesc,CityName,PopulationCount,GeoLocation,CityFIPS
56,Alabama,Birmingham,212237,"(33.5275663773, -86.7988174678)",107000.0
2851,Alabama,Hoover,81619,"(33.3767602729, -86.8051937568)",135896.0
3713,Alabama,Huntsville,180105,"(34.6989692671, -86.6387042882)",137000.0
5549,Alabama,Mobile,195111,"(30.6776248648, -88.1184482714)",150000.0
7779,Alabama,Montgomery,205764,"(32.3472645333, -86.2677059552)",151000.0


# Add Weather

In [29]:
# Get Weather Loop

# Build partial query URL
url = "https://api.wolframalpha.com/v2/query?format=plaintext&output=JSON&input="


print("Beginning Data Retrieval")
print("------------------------")

row_count = 1

# Make Weather DataFrame
weather_df = []
weather_df = pd.DataFrame()
weather_df["Average Temperature"] = ""
weather_df["Historic Low Temp"] = ""
weather_df["Average High Temp"] = ""
weather_df["Average Low Temp"] = ""
weather_df["Historic High Temp"] = ""

for index, row in just_cities.iterrows():
    city_query = row['CityName']
    state_query = row['StateDesc']
    print(f"This is the city query {city_query}, {state_query}")
    try:
        
        ten_year_query = "average+temperature+for+" + city_query.replace(" ","+") + "+" + state_query.replace(" ","+") + "+past+ten+years"

        # Build query URL
        query_url = url + ten_year_query + "&appid=" + wolf_api_key
                       
        # Get Response
        response = requests.get(query_url).json()
                       
        # Obtain, then Append Overall Average Temp
        avg_temp = response["queryresult"]["pods"][1]["subpods"][0]["plaintext"]
        avg_temp_clean = int(avg_temp.split(" °F")[0])
        weather_df.loc[index, "Average Temperature"] = avg_temp_clean

        # Get Temp String (used for all other variables below)
        temp_string = response["queryresult"]["pods"][2]["subpods"][0]["plaintext"]
        
        # Historic Low Temp
        temp_low = temp_string.split(" | ")[2]
        temp_low = temp_low.split("low: ")[1]
        weather_df.loc[index, "Historic Low Temp"] = int(temp_low.split(" °F")[0])

        # Average High Temp
        temp_avghigh = temp_string.split(" | ")[4]
        weather_df.loc[index, "Average High Temp"] = int(temp_avghigh.split(" °F")[0])

        # Average Low Temp
        temp_avglow = temp_string.split(" | ")[5]
        weather_df.loc[index, "Average Low Temp"] = int(temp_avglow.split(" °F")[0])

        # Historic High Temp
        temp_high = temp_string.split(" | ")[6]
        temp_high = temp_high.split("high: ")[1]
        weather_df.loc[index, "Historic High Temp"] = int(temp_high.split(" °F")[0])
        
        # Add Place Holders
        weather_df.loc[index, "State"] = state_query
        weather_df.loc[index, "City"] = city_query
        weather_df.loc[index, "CityFIPS"] = row['CityFIPS']

#         # Temperature Range
#         average_temp_range_list.append(temp_avghigh - temp_avglow)
        
        print(f"Processing Record {row_count} of Set 1 | {city_query}, {state_query}")
        row_count += 1
         
    except Exception as e:
        # TODO Exception Handling needs to be Improved
        print(f"{city_query} not found")
        row_count += 1

weather_df.head()

Beginning Data Retrieval
------------------------
This is the city query Birmingham, Alabama
Processing Record 1 of Set 1 | Birmingham, Alabama
This is the city query Hoover, Alabama
Processing Record 2 of Set 1 | Hoover, Alabama
This is the city query Huntsville, Alabama
Processing Record 3 of Set 1 | Huntsville, Alabama
This is the city query Mobile, Alabama
Processing Record 4 of Set 1 | Mobile, Alabama
This is the city query Montgomery, Alabama
Processing Record 5 of Set 1 | Montgomery, Alabama
This is the city query Tuscaloosa, Alabama
Processing Record 6 of Set 1 | Tuscaloosa, Alabama
This is the city query Anchorage, Alaska
Processing Record 7 of Set 1 | Anchorage, Alaska
This is the city query Avondale, Arizona
Processing Record 8 of Set 1 | Avondale, Arizona
This is the city query Chandler, Arizona
Processing Record 9 of Set 1 | Chandler, Arizona
This is the city query Gilbert, Arizona
Processing Record 10 of Set 1 | Gilbert, Arizona
This is the city query Glendale, Arizona
Pr

Processing Record 86 of Set 1 | Napa, California
This is the city query Newport Beach, California
Processing Record 87 of Set 1 | Newport Beach, California
This is the city query Norwalk, California
Processing Record 88 of Set 1 | Norwalk, California
This is the city query Oakland, California
Processing Record 89 of Set 1 | Oakland, California
This is the city query Oceanside, California
Processing Record 90 of Set 1 | Oceanside, California
This is the city query Ontario, California
Processing Record 91 of Set 1 | Ontario, California
This is the city query Orange, California
Processing Record 92 of Set 1 | Orange, California
This is the city query Oxnard, California
Processing Record 93 of Set 1 | Oxnard, California
This is the city query Palmdale, California
Processing Record 94 of Set 1 | Palmdale, California
This is the city query Pasadena, California
Processing Record 95 of Set 1 | Pasadena, California
This is the city query Perris, California
Processing Record 96 of Set 1 | Perris

Processing Record 171 of Set 1 | Boynton Beach, Florida
This is the city query Cape Coral, Florida
Processing Record 172 of Set 1 | Cape Coral, Florida
This is the city query Clearwater, Florida
Processing Record 173 of Set 1 | Clearwater, Florida
This is the city query Coral Springs, Florida
Processing Record 174 of Set 1 | Coral Springs, Florida
This is the city query Davie, Florida
Processing Record 175 of Set 1 | Davie, Florida
This is the city query Deerfield Beach, Florida
Processing Record 176 of Set 1 | Deerfield Beach, Florida
This is the city query Deltona, Florida
Processing Record 177 of Set 1 | Deltona, Florida
This is the city query Fort Lauderdale, Florida
Processing Record 178 of Set 1 | Fort Lauderdale, Florida
This is the city query Gainesville, Florida
Processing Record 179 of Set 1 | Gainesville, Florida
This is the city query Hialeah, Florida
Processing Record 180 of Set 1 | Hialeah, Florida
This is the city query Hollywood, Florida
Processing Record 181 of Set 1 |

Processing Record 260 of Set 1 | Louisville, Kentucky
This is the city query Baton Rouge, Louisiana
Processing Record 261 of Set 1 | Baton Rouge, Louisiana
This is the city query Kenner, Louisiana
Processing Record 262 of Set 1 | Kenner, Louisiana
This is the city query Lafayette, Louisiana
Processing Record 263 of Set 1 | Lafayette, Louisiana
This is the city query Lake Charles, Louisiana
Processing Record 264 of Set 1 | Lake Charles, Louisiana
This is the city query New Orleans, Louisiana
Processing Record 265 of Set 1 | New Orleans, Louisiana
This is the city query Shreveport, Louisiana
Processing Record 266 of Set 1 | Shreveport, Louisiana
This is the city query Portland, Maine
Processing Record 267 of Set 1 | Portland, Maine
This is the city query Baltimore, Maryland
Processing Record 268 of Set 1 | Baltimore, Maryland
This is the city query Boston, Massachusetts
Processing Record 269 of Set 1 | Boston, Massachusetts
This is the city query Brockton, Massachusetts
Processing Record

Processing Record 346 of Set 1 | Syracuse, New York
This is the city query Yonkers, New York
Yonkers not found
This is the city query Asheville, North Carolin
Asheville not found
This is the city query Cary, North Carolin
Cary not found
This is the city query Charlotte, North Carolin
Charlotte not found
This is the city query Concord, North Carolin
Concord not found
This is the city query Durham, North Carolin
Durham not found
This is the city query Fayetteville, North Carolin
Fayetteville not found
This is the city query Gastonia, North Carolin
Gastonia not found
This is the city query Greensboro, North Carolin
Greensboro not found
This is the city query Greenville, North Carolin
Greenville not found
This is the city query High Point, North Carolin
High Point not found
This is the city query Jacksonville, North Carolin
Jacksonville not found
This is the city query Raleigh, North Carolin
Raleigh not found
This is the city query Wilmington, North Carolin
Wilmington not found
This is the

Processing Record 441 of Set 1 | Midland, Texas
This is the city query Mission, Texas
Processing Record 442 of Set 1 | Mission, Texas
This is the city query Missouri City, Texas
Processing Record 443 of Set 1 | Missouri City, Texas
This is the city query Odessa, Texas
Processing Record 444 of Set 1 | Odessa, Texas
This is the city query Pasadena, Texas
Processing Record 445 of Set 1 | Pasadena, Texas
This is the city query Pearland, Texas
Processing Record 446 of Set 1 | Pearland, Texas
This is the city query Pharr, Texas
Processing Record 447 of Set 1 | Pharr, Texas
This is the city query Plano, Texas
Processing Record 448 of Set 1 | Plano, Texas
This is the city query Richardson, Texas
Processing Record 449 of Set 1 | Richardson, Texas
This is the city query Round Rock, Texas
Processing Record 450 of Set 1 | Round Rock, Texas
This is the city query San Angelo, Texas
Processing Record 451 of Set 1 | San Angelo, Texas
This is the city query San Antonio, Texas
Processing Record 452 of S

,Average Temperature,Historic Low Temp,Average High Temp,Average Low Temp,Historic High Temp,State,City,CityFIPS
56,63.0,11.0,83.0,42.0,104.0,Alabama,Birmingham,107000.0
2851,63.0,11.0,83.0,42.0,104.0,Alabama,Hoover,135896.0
3713,59.0,5.0,81.0,37.0,102.0,Alabama,Huntsville,137000.0
5549,67.0,18.0,84.0,48.0,102.0,Alabama,Mobile,150000.0
7779,64.0,16.0,85.0,45.0,102.0,Alabama,Montgomery,151000.0


# Add second output to Weather Loop so unfound cities are accounted for

In [169]:
# Get Weather Loop

# Build partial query URL
url = "https://api.wolframalpha.com/v2/query?format=plaintext&output=JSON&input="


print("Beginning Data Retrieval")
print("------------------------")

row_count = 1

# Make Weather DataFrame
weather_df = []
weather_df = pd.DataFrame()
weather_df["Average Temperature"] = ""
weather_df["Historic Low Temp"] = ""
weather_df["Average High Temp"] = ""
weather_df["Average Low Temp"] = ""
weather_df["Historic High Temp"] = ""

# Make Weather_Unfound DataFrame
weather_unfound_df = []
weather_unfound_df = pd.DataFrame()
weather_unfound_df["State"] = ""
weather_unfound_df["City"] = ""
weather_unfound_df["CityFIPS"] = ""

for index, row in just_cities.iterrows():
    city_query = row['CityName']
    state_query = row['StateDesc']
    print(f"This is the city query {city_query}, {state_query}")
    try:
        
        ten_year_query = "average+temperature+for+" + city_query.replace(" ","+") + "+" + state_query.replace(" ","+") + "+past+ten+years"

        # Build query URL
        query_url = url + ten_year_query + "&appid=" + wolf_api_key
                       
        # Get Response
        response = requests.get(query_url).json()
                       
        # Obtain, then Append Overall Average Temp
        avg_temp = response["queryresult"]["pods"][1]["subpods"][0]["plaintext"]
        avg_temp_clean = int(avg_temp.split(" °F")[0])
        weather_df.loc[index, "Average Temperature"] = avg_temp_clean

        # Get Temp String (used for all other variables below)
        temp_string = response["queryresult"]["pods"][2]["subpods"][0]["plaintext"]
        
        # Historic Low Temp
        temp_low = temp_string.split(" | ")[2]
        temp_low = temp_low.split("low: ")[1]
        weather_df.loc[index, "Historic Low Temp"] = int(temp_low.split(" °F")[0])

        # Average High Temp
        temp_avghigh = temp_string.split(" | ")[4]
        weather_df.loc[index, "Average High Temp"] = int(temp_avghigh.split(" °F")[0])

        # Average Low Temp
        temp_avglow = temp_string.split(" | ")[5]
        weather_df.loc[index, "Average Low Temp"] = int(temp_avglow.split(" °F")[0])

        # Historic High Temp
        temp_high = temp_string.split(" | ")[6]
        temp_high = temp_high.split("high: ")[1]
        weather_df.loc[index, "Historic High Temp"] = int(temp_high.split(" °F")[0])
        
        # Add Place Holders
        weather_df.loc[index, "State"] = state_query
        weather_df.loc[index, "City"] = city_query
        weather_df.loc[index, "CityFIPS"] = row['CityFIPS']

#         # Temperature Range
#         average_temp_range_list.append(temp_avghigh - temp_avglow)
        
        print(f"Processing Record {row_count} of Set 1 | {city_query}, {state_query}")
        row_count += 1
         
    except Exception as e:
        # TODO Exception Handling needs to be Improved
        print(f"{city_query} not found")
        weather_unfound_df.loc[index, "State"] = state_query
        weather_unfound_df.loc[index, "City"] = city_query
        weather_unfound_df.loc[index, "CityFIPS"] = row['CityFIPS']
        row_count += 1

weather_unfound_df.head()

Beginning Data Retrieval
------------------------
This is the city query Birmingham, Alabama
Processing Record 1 of Set 1 | Birmingham, Alabama
This is the city query Hoover, Alabama
Processing Record 2 of Set 1 | Hoover, Alabama
This is the city query Huntsville, Alabama
Processing Record 3 of Set 1 | Huntsville, Alabama
This is the city query Mobile, Alabama
Processing Record 4 of Set 1 | Mobile, Alabama
This is the city query Montgomery, Alabama
Processing Record 5 of Set 1 | Montgomery, Alabama
This is the city query Tuscaloosa, Alabama
Processing Record 6 of Set 1 | Tuscaloosa, Alabama
This is the city query Anchorage, Alaska
Processing Record 7 of Set 1 | Anchorage, Alaska
This is the city query Avondale, Arizona
Processing Record 8 of Set 1 | Avondale, Arizona
This is the city query Chandler, Arizona
Processing Record 9 of Set 1 | Chandler, Arizona
This is the city query Gilbert, Arizona
Processing Record 10 of Set 1 | Gilbert, Arizona
This is the city query Glendale, Arizona
Pr

Processing Record 87 of Set 1 | Newport Beach, California
This is the city query Norwalk, California
Processing Record 88 of Set 1 | Norwalk, California
This is the city query Oakland, California
Processing Record 89 of Set 1 | Oakland, California
This is the city query Oceanside, California
Processing Record 90 of Set 1 | Oceanside, California
This is the city query Ontario, California
Processing Record 91 of Set 1 | Ontario, California
This is the city query Orange, California
Processing Record 92 of Set 1 | Orange, California
This is the city query Oxnard, California
Processing Record 93 of Set 1 | Oxnard, California
This is the city query Palmdale, California
Processing Record 94 of Set 1 | Palmdale, California
This is the city query Pasadena, California
Processing Record 95 of Set 1 | Pasadena, California
This is the city query Perris, California
Processing Record 96 of Set 1 | Perris, California
This is the city query Pleasanton, California
Processing Record 97 of Set 1 | Pleasan

Processing Record 171 of Set 1 | Boynton Beach, Florida
This is the city query Cape Coral, Florida
Processing Record 172 of Set 1 | Cape Coral, Florida
This is the city query Clearwater, Florida
Processing Record 173 of Set 1 | Clearwater, Florida
This is the city query Coral Springs, Florida
Processing Record 174 of Set 1 | Coral Springs, Florida
This is the city query Davie, Florida
Processing Record 175 of Set 1 | Davie, Florida
This is the city query Deerfield Beach, Florida
Processing Record 176 of Set 1 | Deerfield Beach, Florida
This is the city query Deltona, Florida
Processing Record 177 of Set 1 | Deltona, Florida
This is the city query Fort Lauderdale, Florida
Processing Record 178 of Set 1 | Fort Lauderdale, Florida
This is the city query Gainesville, Florida
Processing Record 179 of Set 1 | Gainesville, Florida
This is the city query Hialeah, Florida
Processing Record 180 of Set 1 | Hialeah, Florida
This is the city query Hollywood, Florida
Processing Record 181 of Set 1 |

Processing Record 260 of Set 1 | Louisville, Kentucky
This is the city query Baton Rouge, Louisiana
Processing Record 261 of Set 1 | Baton Rouge, Louisiana
This is the city query Kenner, Louisiana
Processing Record 262 of Set 1 | Kenner, Louisiana
This is the city query Lafayette, Louisiana
Processing Record 263 of Set 1 | Lafayette, Louisiana
This is the city query Lake Charles, Louisiana
Processing Record 264 of Set 1 | Lake Charles, Louisiana
This is the city query New Orleans, Louisiana
Processing Record 265 of Set 1 | New Orleans, Louisiana
This is the city query Shreveport, Louisiana
Processing Record 266 of Set 1 | Shreveport, Louisiana
This is the city query Portland, Maine
Processing Record 267 of Set 1 | Portland, Maine
This is the city query Baltimore, Maryland
Processing Record 268 of Set 1 | Baltimore, Maryland
This is the city query Boston, Massachusetts
Processing Record 269 of Set 1 | Boston, Massachusetts
This is the city query Brockton, Massachusetts
Processing Record

Yonkers not found
This is the city query Asheville, North Carolin
Asheville not found
This is the city query Cary, North Carolin
Cary not found
This is the city query Charlotte, North Carolin
Charlotte not found
This is the city query Concord, North Carolin
Concord not found
This is the city query Durham, North Carolin
Durham not found
This is the city query Fayetteville, North Carolin
Fayetteville not found
This is the city query Gastonia, North Carolin
Gastonia not found
This is the city query Greensboro, North Carolin
Greensboro not found
This is the city query Greenville, North Carolin
Greenville not found
This is the city query High Point, North Carolin
High Point not found
This is the city query Jacksonville, North Carolin
Jacksonville not found
This is the city query Raleigh, North Carolin
Raleigh not found
This is the city query Wilmington, North Carolin
Wilmington not found
This is the city query Winston-Salem, North Carolin
Winston-Salem not found
This is the city query Fargo

Processing Record 442 of Set 1 | Mission, Texas
This is the city query Missouri City, Texas
Processing Record 443 of Set 1 | Missouri City, Texas
This is the city query Odessa, Texas
Processing Record 444 of Set 1 | Odessa, Texas
This is the city query Pasadena, Texas
Processing Record 445 of Set 1 | Pasadena, Texas
This is the city query Pearland, Texas
Processing Record 446 of Set 1 | Pearland, Texas
This is the city query Pharr, Texas
Processing Record 447 of Set 1 | Pharr, Texas
This is the city query Plano, Texas
Processing Record 448 of Set 1 | Plano, Texas
This is the city query Richardson, Texas
Processing Record 449 of Set 1 | Richardson, Texas
This is the city query Round Rock, Texas
Processing Record 450 of Set 1 | Round Rock, Texas
This is the city query San Angelo, Texas
Processing Record 451 of Set 1 | San Angelo, Texas
This is the city query San Antonio, Texas
Processing Record 452 of Set 1 | San Antonio, Texas
This is the city query Sugar Land, Texas
Processing Record 4

,State,City,CityFIPS
32936,Arizona,Scottsdale,465000
45602,California,Alameda,600562
122603,California,Menifee,646842
156657,California,San Buenaventura (Ventura),665042
187639,California,South Gate,673080


In [185]:
weather_unfound_df.head()

,State,City,CityFIPS
32936,Arizona,Scottsdale,465000
45602,California,Alameda,600562
122603,California,Menifee,646842
156657,California,San Buenaventura (Ventura),665042
187639,California,South Gate,673080


In [170]:
weather_df.to_csv("output/weather_df2.csv")
weather_unfound_df.to_csv("output/weather_unfound_df.csv")

# Census Data

In [135]:
from census import Census
from us import states
import gmaps

# Census & gmaps API Keys
from config import (census_api_key, gkey)
c = Census(census_api_key, year=2013)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [6]:
# What is Available
# pprint(c.acs5.tables())

# Figuring out how to manipulate CityFIPS

In [136]:

# US places
# (example: Fresno city in California is "27000", or prefixed with the state code "0627000")
# state_place(fields, state_fips, place)


# Address	2630 CAHABA RD, BIRMINGHAM, AL, 35223
# MSA/MD Code	13820
# State Code	01
# County Code	073
# Tract Code	0108.02
# MSA/MD Name	BIRMINGHAM-HOOVER, AL
# State Name	ALABAMA
# County Name	JEFFERSON COUNTY

# In the CSV = 0107000-01073010802
# so State code = 01, Place Code = 07000





# # state_county_tract Specification
# state_county_tract(fields, state_fips, county_fips, tract)

# #The geographic name for all census tracts for county 170 in Alaska:
# c.sf1.get('NAME', geo={'for': 'tract:*',
#                        'in': 'state:{} county:170'.format(states.AK.fips)})

# #The same call using the state_county_tract convenience method:
# c.sf1.state_county_tract('NAME', states.AK.fips, '170', Census.ALL)



# #The Format To Use
# state_place(fields, state_fips, place)


# Make Example DataFrame
# example_df = pd.DataFrame({
#     "CityFIPS": ["107000.0","0135896.0","0137000.0"]
# })
# example_df.head()



# Copy DataFrame
census_cities_df = just_cities
# census_cities_df["State ID"] = ""
state_ID_series = []
place_ID_series = []

# Create State and Place IDs from CityFIPS
# for index, row in example_df.iterrows():
for index, row in census_cities_df.iterrows():
    city_FIPS = row['CityFIPS']
    city_FIPS = str(city_FIPS)
#     print(city_FIPS)
    
    if len(city_FIPS)==9:
        state_ID = city_FIPS[0:2]
#         state_ID_series.append(state_ID)
        place_ID = city_FIPS[2:]
    elif len(city_FIPS)==8:
        state_ID = city_FIPS[0:1]
        state_ID = "0" + state_ID
        place_ID = city_FIPS[1:]

#     print(state_ID_series)
#     print(state_ID)
    state_ID_series.append(state_ID)
#     print(place_ID)
    place_ID_series.append(place_ID)
#     print("")
    
# Add Result to DataFrame
census_cities_df["State ID"] = state_ID_series
census_cities_df["Place ID"] = place_ID_series

# Make Smaller DataFrame for testing
census_cities_df_head = census_cities_df.head()
census_cities_df.head()
    

,StateDesc,CityName,PopulationCount,GeoLocation,CityFIPS,State ID,Place ID
56,Alabama,Birmingham,212237,"(33.5275663773, -86.7988174678)",107000.0,01,07000.0
2851,Alabama,Hoover,81619,"(33.3767602729, -86.8051937568)",135896.0,01,35896.0
3713,Alabama,Huntsville,180105,"(34.6989692671, -86.6387042882)",137000.0,01,37000.0
5549,Alabama,Mobile,195111,"(30.6776248648, -88.1184482714)",150000.0,01,50000.0
7779,Alabama,Montgomery,205764,"(32.3472645333, -86.2677059552)",151000.0,01,51000.0


# Census API - Single input

In [125]:
census_data_test_single = c.acs5.state_place('NAME','01','07000')
census_pd_test = pd.DataFrame(census_data_test_single)

census_data_test = c.acs5.state_place(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"),'01','07000')

# Convert to DataFrame
census_pd = pd.DataFrame(census_data_test)
# census_pd.head()

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

census_pd.head()

,State,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,01,"Birmingham city, Alabama",212295.0,35.9,31445.0,19650.0,62258.0,29.326173,7.252644


# Census API Loop

# Don't run this cell for a second time

In [183]:
# # Proof one data submission works
# census_data_test_single = c.acs5.state_place('NAME','01','07000')
# census_pd_test = pd.DataFrame(census_data_test_single)

# census_data_test = []
census_results_df = []

census_results_df = census_cities_df_head

census_data_test_append = []
census_data_test_fromrecords = []
census_data_test_fromdict = []
census_data_test_regular = []

for index, row in census_cities_df.iterrows():
# for index, row in census_cities_df_head.iterrows():
    state_id_query = row['State ID']
    place_id_query = row['Place ID']
    place_id_query = place_id_query.replace(".0","")
    
    
    # The main data submission
    census_data_test = c.acs5.state_place(("NAME", "B19013_001E", "B01003_001E", "B01002_001E", "B19301_001E", "B17001_002E", "B23025_005E"),state_id_query,place_id_query)

#     # Code just to prove the loop works
#     census_data_test.append([state_id_query, place_id_query])

    # Append new record to new variable to build list
    census_data_test_append.append(census_data_test)
    
    # Print to confirm workflow
    print(state_id_query)
    print(place_id_query)
#     print(census_data_test_append)


01
07000
01
35896
01
37000
01
50000
01
51000
01
77256
02
03000
04
04720
04
12000
04
27400
04
27820
04
46000
04
54050
04
55000
04
65000
04
71510
04
73000
04
77000
04
85540
05
23290
05
24550
05
35710
05
41000
05
66080
06
00562
06
00884
06
02000
06
02252
06
02364
06
03526
06
03666
06
04982
06
06000
06
08786
06
08954
06
11194
06
11530
06
13014
06
13210
06
13214
06
13392
06
13588
06
14218
06
15044
06
16000
06
16350
06
16532
06
17918
06
19766
06
21712
06
22020
06
22230
06
22804
06
23182
06
24638
06
24680
06
26000
06
27000
06
28000
06
29000
06
30000
06
32548
06
33000
06
33182
06
33434
06
36000
06
36448
06
36546
06
36770
06
39496
06
39892
06
40130
06
41992
06
43000
06
44000
06
44574
06
45484
06
46842
06
46898
06
47766
06
48256
06
48354
06
49270
06
49670
06
50076
06
50258
06
51182
06
52526
06
53000
06
53322
06
53896
06
53980
06
54652
06
55156
06
56000
06
56700
06
57792
06
58072
06
59451
06
59920
06
59962
06
60018
06
60102
06
60466
06
60620
06
62000
06
62938
06
64000
06
64224
06
65000
06
65042
0

AttributeError: 'list' object has no attribute 'head'

In [184]:
print(census_data_test_append)

[[{'NAME': 'Birmingham city, Alabama', 'B19013_001E': 31445.0, 'B01003_001E': 212295.0, 'B01002_001E': 35.9, 'B19301_001E': 19650.0, 'B17001_002E': 62258.0, 'B23025_005E': 15397.0, 'state': '01', 'place': '07000'}], [{'NAME': 'Hoover city, Alabama', 'B19013_001E': 75020.0, 'B01003_001E': 82264.0, 'B01002_001E': 36.7, 'B19301_001E': 38457.0, 'B17001_002E': 5537.0, 'B23025_005E': 2795.0, 'state': '01', 'place': '35896'}], [{'NAME': 'Huntsville city, Alabama', 'B19013_001E': 48881.0, 'B01003_001E': 182317.0, 'B01002_001E': 36.7, 'B19301_001E': 30916.0, 'B17001_002E': 29494.0, 'B23025_005E': 11036.0, 'state': '01', 'place': '37000'}], [{'NAME': 'Mobile city, Alabama', 'B19013_001E': 38644.0, 'B01003_001E': 195116.0, 'B01002_001E': 35.5, 'B19301_001E': 23385.0, 'B17001_002E': 43928.0, 'B23025_005E': 12229.0, 'state': '01', 'place': '50000'}], [{'NAME': 'Montgomery city, Alabama', 'B19013_001E': 43702.0, 'B01003_001E': 204760.0, 'B01002_001E': 34.2, 'B19301_001E': 24365.0, 'B17001_002E': 447